In [47]:
import bs4
import requests 
import re
import pandas as pd

In [48]:
url = 'https://leagueoflegends.fandom.com/wiki/List_of_champions'
req = requests.get(url)
req = bs4.BeautifulSoup(req.text)

In [49]:
champion_names = [g['data-sort-value'] for g in req.find_all('td', {'data-sort-value': True})][0:-1:3]

In [55]:
def champ_info(champs):

    names = []
    classes = []
    legacies = []
    lanes = []
    resources = []
    adaptives = []
    difficulties = []
    abilities = []

    for champ in champs: 
        url = f"https://leagueoflegends.fandom.com/wiki/{champ}/LoL"
        request = requests.get(url)
        page = bs4.BeautifulSoup(request.text)

        names.append(champ)

        #infobox 
        stats = page.find('div',attrs={'id':'infobox-champion-container'} )

        #extracting class
        class_ = stats.find('div', attrs={'data-source':'role'})
        class_ = class_.find_all('a', class_='mw-redirect')
        class_ = [a.text.strip() for a in class_ if a.text.strip()]
        classes.append(class_)


        #extracting legacy 
        legacy = stats.find('div', attrs={'data-source':'legacy'})
        legacy = [a.text for a in legacy.find_all('a', attrs= {'class': 'mw-redirect'})]
        legacies.append(legacy)

        #extract postition 
        position = stats.find('div', attrs={'data-source':'position'})
        positions = [span.get('data-tip') for span in position.find_all('span', {'data-tip': True})]
        lanes.append(positions)  # Append extracted positions list

        #extract resource 
        resource = stats.find('div', attrs={'data-source':'resource'})
        resource = resource.find('span', {'data-tip': True}).get('data-tip')
        resources.append(resource)

        #extract adaptive 
        adaptive = stats.find('div', attrs={'data-source':'adaptivetype'})
        adaptive = adaptive.find_all('a')[1].text
        adaptives.append(adaptive)

        #extract difficulty 
        difficulty = stats.find('div', attrs={'data-source':'difficulty'})
        difficulty = difficulty.find('div', attrs = {'title': True}).get('title')
        match = re.search(r'(\d+)', difficulty)  # Extract numerical value
        difficulty_level = int(match.group(1))
        difficulties.append(difficulty_level)

        #extract all abiilities
        champ_abilities = ""
        for ability in page.find_all('div', attrs={'class' : 'skill_header skill_wrapper'}):
            texts = [p.text for p in ability.find_all('p')]
            joined = " ".join(texts)
            champ_abilities += joined + " "
        abilities.append(champ_abilities)
   
   
        




    return {'Name' : names, 'Class(es)' :classes, 'Legacy' : legacies, 'Lane' : lanes, 'Resource': resources
            , 'Adaptive' : adaptives, 'Difficulty' : difficulties, 'Abilities': abilities}


data = champ_info(champion_names)
data


{'Name': ['Aatrox',
  'Ahri',
  'Akali',
  'Akshan',
  'Alistar',
  'Ambessa',
  'Amumu',
  'Anivia',
  'Annie',
  'Aphelios',
  'Ashe',
  'Aurelion Sol',
  'Aurora',
  'Azir',
  'Bard',
  "Bel'Veth",
  'Blitzcrank',
  'Brand',
  'Braum',
  'Briar',
  'Caitlyn',
  'Camille',
  'Cassiopeia',
  "Cho'Gath",
  'Corki',
  'Darius',
  'Diana',
  'Dr. Mundo',
  'Draven',
  'Ekko',
  'Elise',
  'Evelynn',
  'Ezreal',
  'Fiddlesticks',
  'Fiora',
  'Fizz',
  'Galio',
  'Gangplank',
  'Garen',
  'Gnar',
  'Gragas',
  'Graves',
  'Gwen',
  'Hecarim',
  'Heimerdinger',
  'Hwei',
  'Illaoi',
  'Irelia',
  'Ivern',
  'Janna',
  'Jarvan IV',
  'Jax',
  'Jayce',
  'Jhin',
  'Jinx',
  "K'Sante",
  "Kai'Sa",
  'Kalista',
  'Karma',
  'Karthus',
  'Kassadin',
  'Katarina',
  'Kayle',
  'Kayn',
  'Kennen',
  "Kha'Zix",
  'Kindred',
  'Kled',
  "Kog'Maw",
  'LeBlanc',
  'Lee Sin',
  'Leona',
  'Lillia',
  'Lissandra',
  'Lucian',
  'Lulu',
  'Lux',
  'Malphite',
  'Malzahar',
  'Maokai',
  'Master Yi',
  '

In [56]:
df = pd.DataFrame(data)
df.to_csv('champs-stats.csv')
df

,Name,Class(es),Legacy,Lane,Resource,Adaptive,Difficulty,Abilities
0,Aatrox,[Juggernaut],"[Fighter, Tank]",[Top],Manaless,Physical,2,"Innate: Periodically, Aatrox empowers his next..."
1,Ahri,[Burst],"[Mage, Assassin]",[Middle],Mana,Magic,2,Innate: Ahri generates a stack of Essence Frag...
2,Akali,[Assassin],[Assassin],"[Top, Middle]",Energy,Physical,2,Innate: When Akali damages an enemy champion ...
3,Akshan,"[Marksman, Assassin]","[Marksman, Assassin]",[Middle],Mana,Physical,3,"Innate: Whenever Akshan uses a basic attack, h..."
4,Alistar,[],"[Tank, Support]",[Support],Mana,Magic,1,Innate: Alistar generates a stack of Triumph f...
...,...,...,...,...,...,...,...,...
165,Zeri,[Marksman],[Marksman],[Bottom],Mana,Physical,2,Innate: Zeri generates 1 charge for every 40 u...
166,Ziggs,[Artillery],[Mage],"[Bottom, Middle]",Mana,Magic,2,"Innate: Periodically, Ziggs empowers his next ..."
167,Zilean,[Specialist],"[Support, Mage]",[Support],Mana,Magic,2,Innate: Zilean generates Champions killed wit...
168,Zoe,[Burst],"[Mage, Support]",[Middle],Mana,Magic,3,"Innate: After casting an ability, Zoe empowers..."
